In [1]:
# -*- encoding: utf-8 -*-
from collections import namedtuple
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import scipy.ndimage as ndimage
import scipy.ndimage.filters as filters
import scipy.optimize as optimize
import uncertainties
from uncertainties import ufloat, umath
import uncertainties.unumpy as unp
from uncertainties.unumpy import nominal_values as noms
from textable import table
import glob as glob

# bypass UsageError when there is no GUI
try:
    %matplotlib inline
except:
    pass

plt.rcParams['image.cmap'] = 'viridis'
plt.rcParams['figure.figsize'] = (16, 10)
plt.rcParams['font.size'] = 16
plt.style.use('ggplot')

In [2]:
# Constants

In [3]:
DELTA_LENGTH = 0.01 * 1e-3
RADIUS_HOLE = ufloat(9 * 1e-3, DELTA_LENGTH) / 2
RADIUS = 57.3 * 1e-3
LAMBDA_1 = 1.54093 * 1e-10
LAMBDA_2 = 1.54478 * 1e-10
LAMBDA = 1.5417 * 1e-10

In [4]:
# Fit Function

In [5]:
def func(x, m, b):
    return m * x + b

In [6]:
# Calculate Function

In [7]:
def _calculate_difference(array, arrayd, arraynorm):
    # Unused lists, but maybe for later 
    listMMin = []
    listMin = []
    listHKL = []
    listM = []
    # Used lists
    listMMinN = []
    listMinN = []
    listHKLN = []
    listMN = []
    # Find unique D Values
    arrUniqueD = np.unique(array['d'])
    # Loop over all d values
    for idxd, dnorm, d in zip(range(len(arrayd)), arraynorm, arrayd):
        # Value of d to compare with the values of the formfactor table
        perD = 1 / (2 * d.nominal_value * 1e8)
        # Find best matching perD values
        intDBest = 0
        fltAbsMin = 100
        for perd in arrUniqueD:
            if abs(perd - perD) < fltAbsMin:
                fltAbsMin = abs(perd - perD)
                intDBest = perd
        # Get rows to use for comparison
        arrayM = array[array['d'] == intDBest]
        
        # Compare values
        # Unused
        fltDeltaMin = 100
        idxMMin = 0
        varHKL = []
        # Used
        idxMMinN = 0
        fltDeltaMinN = 100
        varHKLN = []
        # Loop over all rows for comparison
        for idxm, dm in enumerate(arrayM['deltam']):
            # difference in percent
            delta = abs(dm - dnorm) / dm
            # If delta is smaller then the minumum and the combination of h,k,l is not used
            if delta < fltDeltaMinN:
                if [arrayM[idxm]['h'], arrayM[idxm]['k'], arrayM[idxm]['l']] not in listHKLN:
                    fltDeltaMinN = delta
                    idxMMinN = idxm
                    mN = dm
                    varHKLN = [arrayM[idxm]['h'], arrayM[idxm]['k'], arrayM[idxm]['l']]
            # If delta is just smaller
            if delta < fltDeltaMin:
                fltDeltaMin = delta
                idxMMin = idxm
                m = dm
                varHKL = [array[idxMMin]['h'], array[idxMMin]['k'], array[idxMMin]['l']]
        # Unused
        listMin.append(fltDeltaMin * 100)
        listMMin.append(idxMMin)
        listHKL.append(varHKL)
        listM.append(m)
        # Used
        listMinN.append(fltDeltaMinN * 100)
        listMMinN.append(idxMMinN)
        listHKLN.append(varHKLN)
        listMN.append(mN)
    return len(np.unique(listMMin)), sum(listMin) / len(listMin), listHKL, listMin, listM,\
        len(np.unique(listMMinN)), sum(listMinN) / len(listMinN), listHKLN, listMinN, listMN

In [8]:
# Load Measured Data

In [9]:
arrProbeOneRaw = np.genfromtxt('raw/probe6_1.dat', unpack=True)
arrProbeTwoRaw = np.genfromtxt('raw/probe6_2.dat', unpack=True)
arrSaltOne = np.genfromtxt('raw/salt1_1.dat', unpack=True)
arrSaltTwo = np.genfromtxt('raw/salt1_2.dat', unpack=True)

In [10]:
# Build the sum of the first and second, third and forth for the probe

In [11]:
arrProbeOne = np.copy(arrProbeOneRaw[2:])
arrProbeOne[0] = (arrProbeOneRaw[0] + arrProbeOneRaw[1]) / 2
arrProbeOne[1] = (arrProbeOneRaw[2] + arrProbeOneRaw[3]) / 2
arrProbeTwo = np.copy(arrProbeTwoRaw[:-2])
arrProbeTwo[-1] = (arrProbeTwoRaw[-1] + arrProbeTwoRaw[-2]) / 2
arrProbeTwo[-2] = (arrProbeTwoRaw[-2] + arrProbeTwoRaw[-3]) / 2

In [12]:
# Load HKL and values for calculations

In [13]:
listFileHKL = glob.glob('build/hkl_one/*')
listDType = [
    ('h', '<i8'),
    ('k', '<i8'),
    ('l', '<i8'),
    ('m', '<f8'),
    ('deltam', '<f8'),
    ('d', '<f8')
]
listArrHKL = []
listArrName = []
listArrPlot = [[], []]

for path in listFileHKL:
    file = path.replace('\\', '/').split('/')[-1]
    name, typ = file.split('.')
    name = name.replace('Caesiumstruktur', 'Caesiumchlorid')
    listArrName.append(name)
    listArrHKL.append(np.sort(np.genfromtxt(path, dtype=listDType), order='m'))
    listArrPlot[0].append(name)
    listArrPlot[1].append(np.sort(np.genfromtxt(path, dtype=listDType), order='m'))
listArrPlot[0].append('Steinsalz')
array = np.sort(np.genfromtxt('build/hkl_two/Steinsalz_Chlor_Calcium.dat', dtype=listDType), order='m')
listArrPlot[1].append(array[array['d'] == 20][:-16])
listArrPlot[0].append('Caesiumchlorid')
array = np.sort(np.genfromtxt('build/hkl_two/Caesiumstruktur_Caesium_Chlor.dat', dtype=listDType), order='m')
listArrPlot[1].append(array[array['d'] == 0])

In [14]:
# List with the hkl values

In [15]:
listHKL = []
for idx, arrhkl in enumerate(listArrPlot[1]):
    listUsed = []
    listHKL.append([])
    for hkl in arrhkl:
        if [hkl[0], hkl[1], hkl[2]] not in listUsed:    
            listHKL[idx].append(r'\text{{({0} {1} {2})}}'.format(hkl[0], hkl[1], hkl[2]))
            listUsed.append([hkl[0], hkl[1], hkl[2]])

In [16]:
# Write table of hkl

In [17]:
with open('build/tex/hkl.tex', 'w') as f:
    f.write(
        table(
            listArrPlot[0],
            listHKL
        )
    )

In [18]:
# Load HKL and values for calculations for presentation

In [19]:
listDType = [
    ('h', '<i8'),
    ('k', '<i8'),
    ('l', '<i8'),
    ('m', '<f8'),
    ('deltam', '<f8'),
    ('d', '<f8')
]
idxStein = 0
idxFluor = 1
idxZink = 2
idxCaes = 3
listNames = ['Steinsalz', 'Fluorid', 'Zinkblende', 'Caesiumstruktur']
listArrHKLTwo = []
listArrNameTwo = []
listArrHKLTwoSum = []
listArrNameTwoSum = []

for idx, name in enumerate(listNames):
    listFileTwo = glob.glob('build/hkl_two/{0}*'.format(name))
    listArrHKLTwo.append([])
    listArrNameTwo.append([])

    for path in listFileTwo:
        file = path.replace('\\', '/').split('/')[-1]
        name, typ = file.split('.')
        name = name.replace('Caesiumstruktur', 'Caesiumchlorid')
        listArrNameTwo[idx].append(name)
        listArrNameTwoSum.append(name)
        listArrHKLTwoSum.append(np.sort(np.genfromtxt(path, dtype=listDType), order='m'))
        listArrHKLTwo[idx].append(np.sort(np.genfromtxt(path, dtype=listDType), order='m'))

In [20]:
# Create lists for output

In [21]:
listHKLTwo = []
listHKLName = []
intSubs = 5
for idx, name in enumerate(listNames):
    listHKLTwo.append([[] for i in range(5)])
    listHKLName.append([[] for i in range(5)])
    for idx3, nam in enumerate(listArrName):
        if nam in name:
            break
    listHKLName[idx][0].append(name)
    listHKLName[idx][1].append(name)
    listHKLName[idx][2].append(name)
    listHKLName[idx][3].append(name)
    listHKLName[idx][4].append(name)
    listHKLTwo[idx][0].append(listHKL[idx3])
    listHKLTwo[idx][1].append(listHKL[idx3])
    listHKLTwo[idx][2].append(listHKL[idx3])
    listHKLTwo[idx][3].append(listHKL[idx3])
    listHKLTwo[idx][4].append(listHKL[idx3])
    for idx2, arrhkl in enumerate(listArrHKLTwo[idx]):
        arrhkl = arrhkl[arrhkl['d'] == 0]
        
        if idx2 < len(listArrHKLTwo[idx]) * 1 / 5:
            listHKLTwo[idx][4].append([r'\text{{({0} {1} {2})}}'.format(hkl[0], hkl[1], hkl[2]) for hkl in arrhkl])
            listHKLName[idx][4].append(r'{0}/{1}'.format(*listArrNameTwo[idx][idx2].split('_')[1:]))
            
        elif idx2 < len(listArrHKLTwo[idx]) * 2 / 5:
            listHKLTwo[idx][3].append([r'\text{{({0} {1} {2})}}'.format(hkl[0], hkl[1], hkl[2]) for hkl in arrhkl])
            listHKLName[idx][3].append(r'{0}/{1}'.format(*listArrNameTwo[idx][idx2].split('_')[1:]))
            
        elif idx2 < len(listArrHKLTwo[idx]) * 3 / 5:
            listHKLTwo[idx][2].append([r'\text{{({0} {1} {2})}}'.format(hkl[0], hkl[1], hkl[2]) for hkl in arrhkl])
            listHKLName[idx][2].append(r'{0}/{1}'.format(*listArrNameTwo[idx][idx2].split('_')[1:]))
            
        elif idx2 < len(listArrHKLTwo[idx]) * 4 / 5:
            listHKLTwo[idx][1].append([r'\text{{({0} {1} {2})}}'.format(hkl[0], hkl[1], hkl[2]) for hkl in arrhkl])
            listHKLName[idx][1].append(r'{0}/{1}'.format(*listArrNameTwo[idx][idx2].split('_')[1:]))
            
        else:
            listHKLTwo[idx][0].append([r'\text{{({0} {1} {2})}}'.format(hkl[0], hkl[1], hkl[2]) for hkl in arrhkl])
            listHKLName[idx][0].append(r'{0}/{1}'.format(*listArrNameTwo[idx][idx2].split('_')[1:]))

In [22]:
# Write output for hkl

In [23]:
listSuffix = ['first', 'second', 'third', 'fourth', 'fifth']
for idx, name in enumerate(listNames):
    for idx2, suffix in enumerate(listSuffix):
        with open('build/tex/{0}_{1}.tex'.format(name, suffix), 'w') as f:
            f.write(
                table(
                    listHKLName[idx][idx2],
                    listHKLTwo[idx][idx2]
                )
            )

In [24]:
# use meter instead of mm

In [25]:
arrProbeOne *= 1e-3
arrProbeTwo *= 1e-3
arrSaltOne *= 1e-3
arrSaltTwo *= 1e-3

In [26]:
# uncertainties <3

In [27]:
uarrProbeOne = unp.uarray(arrProbeOne, DELTA_LENGTH)
uarrProbeTwo = unp.uarray(arrProbeTwo, DELTA_LENGTH)
uarrSaltOne = unp.uarray(arrSaltOne, DELTA_LENGTH)
uarrSaltTwo = unp.uarray(arrSaltTwo, DELTA_LENGTH)

In [28]:
# Add hole radius

In [29]:
uarrProbeOne += RADIUS_HOLE
uarrProbeTwo += RADIUS_HOLE
uarrSaltOne += RADIUS_HOLE
uarrSaltTwo += RADIUS_HOLE

In [30]:
# Combination of the arrays one and two

In [31]:
uarrProbe = np.append(uarrProbeOne, 180 * 1e-3 - uarrProbeTwo)
uarrProbeInv = np.append( 180 * 1e-3 - uarrProbeOne, uarrProbeTwo)[::-1]
uarrSalt = np.append(uarrSaltOne, 180 * 1e-3 - uarrSaltTwo)
uarrSaltInv = np.append(180 * 1e-3 - uarrSaltOne, uarrSaltTwo)[::-1]

In [32]:
# Winkel in rad

In [33]:
uarrThetaProbe = uarrProbe / RADIUS
uarrThetaProbeInv = uarrProbeInv / RADIUS
uarrThetaSalt = uarrSalt / RADIUS
uarrThetaSaltInv = uarrSaltInv / RADIUS

In [34]:
# Measured 2 Theta, so theta is the half

In [35]:
uarrThetaProbe /= 2
uarrThetaSalt /= 2
uarrThetaProbeInv /= 2
uarrThetaSaltInv /= 2

In [36]:
# Calculate Bragg

In [37]:
uarrDProbe = LAMBDA / (2 * unp.sin(uarrThetaProbe))
uarrDSalt = LAMBDA / (2 * unp.sin(uarrThetaSalt))
uarrDProbeInv = LAMBDA / (2 * unp.sin(uarrThetaProbeInv))
uarrDSaltInv = LAMBDA / (2 * unp.sin(uarrThetaSaltInv))

In [38]:
# Norm D

In [39]:
uarrDNormProbe = uarrDProbe[0] / uarrDProbe
uarrDNormSalt = uarrDSalt[0] / uarrDSalt
uarrDNormProbeInv = uarrDProbeInv[0] / uarrDProbeInv
uarrDNormSaltInv = uarrDSaltInv[0] / uarrDSaltInv

In [40]:
# Lists of measured data

In [41]:
listData = [
    [
        uarrSalt * 1e3,
        uarrThetaSalt * 360 / np.pi,
        uarrThetaSalt * 180 / np.pi,
        uarrDSalt * 1e10
    ],
    [
        uarrSaltInv * 1e3,
        uarrThetaSaltInv * 360 / np.pi,
        uarrThetaSaltInv * 180 / np.pi,
        uarrDSaltInv * 1e10
    ],
    [
        uarrProbe * 1e3,
        uarrThetaProbe * 360 / np.pi,
        uarrThetaProbe * 180 / np.pi,
        uarrDProbe * 1e10
    ],
    [
        uarrProbeInv * 1e3,
        uarrThetaProbeInv * 360 / np.pi,
        uarrThetaProbeInv * 180 / np.pi,
        uarrDProbeInv * 1e10
    ]
]
listDataNames = [
    'data_salt',
    'data_salt_inv',
    'data_probe',
    'data_probe_inv'
]

In [42]:
# Write Table of measured data

In [43]:
for name, lists in zip(listDataNames, listData):
    with open('build/tex/{0}.tex'.format(name), 'w') as f:
        f.write(
                table(
                    [
                        r'$s\,/\,\si{\milli\meter}$',
                        r"$\Theta'\,/\,\si{\degree}$",
                        r'$\Theta\,/\,\si{\degree}$',
                        r'$d\,/\,\si{\angstrom}$'
                    ],
                    [
                        lists[0],
                        lists[1],
                        lists[2],
                        lists[3]
                    ]
                    )
                )

In [44]:
# Compare D Values to the values of HKL for probe

In [45]:
listDType = [
    ('name', np.str_, 200),
    ('uniquem', '<f8'),
    ('average', 'O'),
    ('difference', 'O'),
    ('dm', 'O'),
    ('hkl', 'O')
]
arrProbeDiff = np.empty(len(listArrHKL), dtype=listDType)
arrSaltDiff = np.empty(len(listArrHKLTwoSum), dtype=listDType)
arrProbeDiffInv = np.empty(len(listArrHKL), dtype=listDType)
arrSaltDiffInv = np.empty(len(listArrHKLTwoSum), dtype=listDType)
arrProbeDiffN = np.empty(len(listArrHKL), dtype=listDType)
arrSaltDiffN = np.empty(len(listArrHKLTwoSum), dtype=listDType)
arrProbeDiffInvN = np.empty(len(listArrHKL), dtype=listDType)
arrSaltDiffInvN = np.empty(len(listArrHKLTwoSum), dtype=listDType)

for idxProbe, array, name in zip(range(len(listArrHKL)), listArrHKL, listArrName):
    # Probe
    values = _calculate_difference(array, uarrDProbe, uarrDNormProbe)
    arrProbeDiff[idxProbe]['name'] = r'\text{{{}}}'.format(name)
    arrProbeDiff[idxProbe]['uniquem'] = values[0]
    arrProbeDiff[idxProbe]['average'] = values[1]
    arrProbeDiff[idxProbe]['hkl'] = values[2]
    arrProbeDiff[idxProbe]['difference'] = values[3]
    arrProbeDiff[idxProbe]['dm'] = values[4]
    arrProbeDiffN[idxProbe]['name'] = r'\text{{{}}}'.format(name)
    arrProbeDiffN[idxProbe]['uniquem'] = values[5]
    arrProbeDiffN[idxProbe]['average'] = values[6]
    arrProbeDiffN[idxProbe]['hkl'] = values[7]
    arrProbeDiffN[idxProbe]['difference'] = values[8]
    arrProbeDiffN[idxProbe]['dm'] = values[9]
    # Probe Inv
    values = _calculate_difference(array, uarrDProbeInv, uarrDNormProbeInv)
    arrProbeDiffInv[idxProbe]['name'] = r'\text{{{}}}'.format(name)
    arrProbeDiffInv[idxProbe]['uniquem'] = values[0]
    arrProbeDiffInv[idxProbe]['average'] = values[1]
    arrProbeDiffInv[idxProbe]['hkl'] = values[2]
    arrProbeDiffInv[idxProbe]['difference'] = values[3]
    arrProbeDiffInv[idxProbe]['dm'] = values[4]
    arrProbeDiffInvN[idxProbe]['name'] = r'\text{{{}}}'.format(name)
    arrProbeDiffInvN[idxProbe]['uniquem'] = values[5]
    arrProbeDiffInvN[idxProbe]['average'] = values[6]
    arrProbeDiffInvN[idxProbe]['hkl'] = values[7]
    arrProbeDiffInvN[idxProbe]['difference'] = values[8]
    arrProbeDiffInvN[idxProbe]['dm'] = values[9]

for idxSalt, array, name in zip(range(len(listArrHKLTwoSum)), listArrHKLTwoSum, listArrNameTwoSum):
    # Salt
    values = _calculate_difference(array, uarrDSalt, uarrDNormSalt)
    arrSaltDiff[idxSalt]['name'] = r'\text{{{}}}'.format(name)
    arrSaltDiff[idxSalt]['uniquem'] = values[0]
    arrSaltDiff[idxSalt]['average'] = values[1]
    arrSaltDiff[idxSalt]['hkl'] = values[2]
    arrSaltDiff[idxSalt]['difference'] = values[3]
    arrSaltDiff[idxSalt]['dm'] = values[4]
    arrSaltDiffN[idxSalt]['name'] = r'\text{{{}}}'.format(name)
    arrSaltDiffN[idxSalt]['uniquem'] = values[5]
    arrSaltDiffN[idxSalt]['average'] = values[6]
    arrSaltDiffN[idxSalt]['hkl'] = values[7]
    arrSaltDiffN[idxSalt]['difference'] = values[8]
    arrSaltDiffN[idxSalt]['dm'] = values[9]
    # Salt Inv
    values = _calculate_difference(array, uarrDSaltInv, uarrDNormSaltInv)
    arrSaltDiffInv[idxSalt]['name'] = r'\text{{{}}}'.format(name)
    arrSaltDiffInv[idxSalt]['uniquem'] = values[0]
    arrSaltDiffInv[idxSalt]['average'] = values[1]
    arrSaltDiffInv[idxSalt]['hkl'] = values[2]
    arrSaltDiffInv[idxSalt]['difference'] = values[3]
    arrSaltDiffInv[idxSalt]['dm'] = values[4]
    arrSaltDiffInvN[idxSalt]['name'] = r'\text{{{}}}'.format(name)
    arrSaltDiffInvN[idxSalt]['uniquem'] = values[5]
    arrSaltDiffInvN[idxSalt]['average'] = values[6]
    arrSaltDiffInvN[idxSalt]['hkl'] = values[7]
    arrSaltDiffInvN[idxSalt]['difference'] = values[8]
    arrSaltDiffInvN[idxSalt]['dm'] = values[9]

In [46]:
# Calculate best 10 matches and write Output Tex data

In [47]:
listArrDiff = [
    arrProbeDiffN, arrProbeDiffInvN,
    arrSaltDiffN, arrSaltDiffInvN
    ]
listNameDiff = [
    'arrProbeDiffN', 'arrProbeDiffInvN',
    'arrSaltDiffN', 'arrSaltDiffInvN'
]
listDType = [
    ('name', np.str_, 200),
    ('uniquem', '<f8'),
    ('average', 'O'),
    ('difference', 'O'),
    ('dm', 'O'),
    ('hkl', 'O')
]

listLaticeAll = []
listThetaAll = []
listSqrtMAll = []
listNumberTable = []

for number, array, name in zip(range(len(listArrDiff)), listArrDiff, listNameDiff):
    arrUniqueM = np.unique(array['uniquem'])[::-1]
    if number < 2:
        arrTopTen = np.empty(4, dtype=listDType)
    else:
        arrTopTen = np.empty(4, dtype=listDType)
    index = 0
    arrNum = array[array['uniquem'] == arrUniqueM[0]]
    arrSort = np.sort(arrNum, order='average')
    for row in arrSort:
        for idx, element in enumerate(row):
            arrTopTen[index][idx] = element
        index += 1
        if index >= 4:
            break
        elif index >= 4 and number < 2:
            break
            
    listNumberTable.append(arrTopTen['name'])
    if number == 0:
        arrDNorm = uarrDNormProbe
        arrD = uarrDProbe
        arrTheta = uarrThetaProbe
    elif number == 1:
        arrDNorm = uarrDNormProbeInv
        arrD = uarrDProbeInv
        arrTheta = uarrThetaProbeInv
    elif number == 2:
        arrDNorm = uarrDNormSalt
        arrD = uarrDSalt
        arrTheta = uarrThetaSalt
    else:
        arrDNorm = uarrDNormSaltInv
        arrD = uarrDSaltInv
        arrTheta = uarrThetaSaltInv

    with open('build/tex/{0}.tex'.format(name), 'w') as f:
        f.write(
            table(
                [
                    r'Kristallstruktur',
                    r'$\bar{\Delta}\,/\,\si{\percent}$'
                ],
                [
                    arrTopTen['name'],
                    arrTopTen['average']
                    ]
                )
            )
    listHKL = [[] for i in range(4)]
    listM = [[] for i in range(4)]
    listHDiff = [[] for i in range(4)]
    listD = [[] for i in range(4)]
    listDNorm = [[] for i in range(4)]
    listTheta = [[] for i in range(4)]
    listA = [[] for i in range(4)]
    listSqrtM = [[] for i in range(4)]
    listCos = [[] for i in range(4)]
    for idx in range(4):
        nameCheck = arrTopTen['name'][idx].replace('\\text{', '').replace('}', '')
        hkl = arrTopTen['hkl'][idx]
        m = arrTopTen['dm'][idx]
        diff = arrTopTen['difference'][idx]
        listUsed = []        
        if number < 2:
            listForNum = listArrName
        else:
            listForNum = listArrNameTwoSum
        for num, nameArr in enumerate(listForNum):
            if nameArr == nameCheck:
                break
        if number < 2:
            listForSearch = listArrHKL[num]
        else:
            listForSearch = listArrHKLTwoSum[num]
        listForSearchHKL = []
        for arrHKL in listForSearch:
            if [arrHKL['h'], arrHKL['k'], arrHKL['l']] not in listForSearchHKL:
                listForSearchHKL.append([arrHKL['h'], arrHKL['k'], arrHKL['l']])
        for arrHKL in listForSearchHKL:
            for nums, count in enumerate(hkl):
                varNotInList = False
                if count not in listUsed:
                    if count == arrHKL:
                        listUsed.append(count)
                        listHKL[idx].append(r'\text{{({0} {1} {2})}}'.format(count[0], count[1], count[2]))
                        listM[idx].append(m[nums])
                        listHDiff[idx].append(diff[nums])
                        listD[idx].append(arrD[nums]*1e10)
                        listDNorm[idx].append(arrDNorm[nums])
                        listTheta[idx].append(arrTheta[nums] * 180 / np.pi)
                        listCos[idx].append(unp.cos(arrTheta[nums])**2)
                        sqrtm = np.sqrt(count[0]**2 + count[1]**2 + count[2]**2)
                        listSqrtM[idx].append(sqrtm)
                        listA[idx].append(arrD[nums] * sqrtm * 1e10)
                        varNotInList = False
                        break
                    else:
                        varNotInList = True
            if len(listUsed) == len(hkl):
                break
            if varNotInList:
                listHKL[idx].append(r'\text{{({0} {1} {2})}}'.format(arrHKL[0], arrHKL[1], arrHKL[2]))
                listM[idx].append('')
                listCos[idx].append('')
                listHDiff[idx].append('')
                listD[idx].append('')
                listDNorm[idx].append('')
                listTheta[idx].append('')
                listA[idx].append('')
                listSqrtM[idx].append('')
    listLaticeAll.append(listA)
    listThetaAll.append(listTheta)
    listSqrtMAll.append(listSqrtM)
    
    with open('build/tex/{0}_first_one.tex'.format(name), 'w') as f:
        f.write(
            table(
                [
                    r'hkl',
                    r'$\frac{d_1}{d_i}$',
                    r'$\sqrt{{\frac{{m_i}}{{m_1}}}}$',
                    r'$\Delta\,/\si{{\percent}}$'
                ],
                [
                    listHKL[0],
                    listDNorm[0],
                    listM[0],
                    listHDiff[0]
                ]                
            )
        )
    with open('build/tex/{0}_first_two.tex'.format(name), 'w') as f:
        f.write(
            table(
                [
                    r'hkl',
                    r'$d_i\,/\,\si{\angstrom}$',
                    r'$\sqrt{m}$',
                    r'$\Theta\,/\,\si{\degree}$',
                    r'$\cos^2{(\Theta)}$',
                    r'$a\,/\,\si{\angstrom}$'
                ],
                [
                    listHKL[0],
                    listD[0],
                    listSqrtM[0],
                    listTheta[0],
                    listCos[0],
                    listA[0]
                ]                
            )
        )
    with open('build/tex/{0}_second_one.tex'.format(name), 'w') as f:
        f.write(
            table(
                [
                    r'hkl',
                    r'$\frac{d_1}{d_i}$',
                    r'$\sqrt{{\frac{{m_i}}{{m_1}}}}$',
                    r'$\Delta\,/\si{{\percent}}$'
                ],
                [
                    listHKL[1],
                    listDNorm[1],
                    listM[1],
                    listHDiff[1]
                ]
                
            )
        )
    with open('build/tex/{0}_second_two.tex'.format(name), 'w') as f:
        f.write(
            table(
                [
                    r'hkl',
                    r'$d_i\,/\,\si{\angstrom}$',
                    r'$\sqrt{m}$',
                    r'$\Theta\,/\,\si{\degree}$',
                    r'$\cos^2{(\Theta)}$',
                    r'$a\,/\,\si{\angstrom}$'
                ],
                [
                    listHKL[1],
                    listD[1],
                    listSqrtM[1],
                    listTheta[1],
                    listCos[1],
                    listA[1]
                ]
                
            )
        )
    with open('build/tex/{0}_third_one.tex'.format(name), 'w') as f:
        f.write(
            table(
                [
                    r'hkl',
                    r'$\frac{d_1}{d_i}$',
                    r'$\sqrt{{\frac{{m_i}}{{m_1}}}}$',
                    r'$\Delta\,/\si{{\percent}}$'
                ],
                [
                    listHKL[2],
                    listDNorm[2],
                    listM[2],
                    listHDiff[2]
                ]
                
            )
        )
    with open('build/tex/{0}_third_two.tex'.format(name), 'w') as f:
        f.write(
            table(
                [
                    r'hkl',
                    r'$d_i\,/\,\si{\angstrom}$',
                    r'$\sqrt{m}$',
                    r'$\Theta\,/\,\si{\degree}$',
                    r'$\cos^2{(\Theta)}$',
                    r'$a\,/\,\si{\angstrom}$'
                ],
                [
                    listHKL[2],
                    listD[2],
                    listSqrtM[2],
                    listTheta[2],
                    listCos[2],
                    listA[2]
                ]
                
            )
        )
    with open('build/tex/{0}_fourth_one.tex'.format(name), 'w') as f:
        f.write(
            table(
                [
                    r'hkl',
                    r'$\frac{d_1}{d_i}$',
                    r'$\sqrt{{\frac{{m_i}}{{m_1}}}}$',
                    r'$\Delta\,/\si{{\percent}}$'
                ],
                [
                    listHKL[3],
                    listDNorm[3],
                    listM[3],
                    listHDiff[3]
                ]
                
            )
        )
    with open('build/tex/{0}_fourth_two.tex'.format(name), 'w') as f:
        f.write(
            table(
                [
                    r'hkl',
                    r'$d_i\,/\,\si{\angstrom}$',
                    r'$\sqrt{m}$',
                    r'$\Theta\,/\,\si{\degree}$',
                    r'$\cos^2{(\Theta)}$',
                    r'$a\,/\,\si{\angstrom}$'
                ],
                [
                    listHKL[3],
                    listD[3],
                    listSqrtM[3],
                    listTheta[3],
                    listCos[3],
                    listA[3]
                ]
                
            )
        )

In [48]:
# Calculate latice constant

In [49]:
listNumber = ['first', 'second', 'third', 'fourth']
listName = [r'Probe,1', r'Probe,2', r'Salz,1', r'Salz,2']
for latice, theta, name, top, probe in zip(listLaticeAll, listThetaAll, listNameDiff, listNumberTable, listName):
    listM = []
    listB = []
    listA = []
    for lat, the, num in zip(latice, theta, listNumber):
        angle = [np.cos(entry.nominal_value * np.pi / 180)**2 for entry in the if isinstance(entry, uncertainties.core.AffineScalarFunc)]
        number = [entry.nominal_value for entry in lat if isinstance(entry, uncertainties.core.AffineScalarFunc)]

        popt, pcov = optimize.curve_fit(func, angle, number)
        x = np.linspace(0, 1, 1e5)
        
        listM.append(ufloat(popt[0], pcov[0][0]))
        listB.append(ufloat(popt[1], pcov[1][1]))
        listA.append(ufloat(popt[1], pcov[1][1]))
        
        plt.plot(angle, number, 'ko', label=r'Messwerte', markersize=10)
        plt.plot(x, func(x, *popt), 'r-', label=r'Regressionsgerade $f(x) = c \cdot x + b$'.format(probe))
        plt.xlabel(r'$\cos^2{(\Theta)}$', fontsize=16)
        plt.ylabel(r'Gitterkonstante / $\AA$', fontsize=16)
        plt.legend(loc='best')
        plt.xlim([0, 1])
        plt.savefig('build/pic/{0}_{1}.pdf'.format(name, num))
        plt.clf()
    
    with open('build/tex/{0}_fit.tex'.format(name), 'w') as f:
        f.write(
            table(
                [
                    r'',
                    r'$c_\text{{{0}}}\,/\,\si{{1\per\angstrom}}$'.format(probe),
                    r'$b\,/\,\si{\angstrom}$',
                    r'$a\,/\,\si{\angstrom}$'
                ],
                [
                    top,
                    listM,
                    listB,
                    listA
                ]
            )
        )